In [1]:
import glob
import time
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# importing machine learning models for prediction
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from vecstack import stacking
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.ensemble import GradientBoostingClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pandas as pd
df=pd.read_csv('sqliv2.csv',encoding='UTF16')
df=df.dropna(subset=['Label'])
df.shape

(33761, 2)

In [3]:
df.tail(10)

,Sentence,Label
33751,gorin.mccracken@evelin.am,0
33752,terrateig,0
33753,24220,0
33754,41,0
33755,yeti llamus,0
33756,syrett,0
33757,arrechea bellveh,0
33758,1664,0
33759,almaluez,0
33760,f6lo40r06,0


In [4]:
df.head(10)

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1
5,select * from users where id = '1' or @ @1 = ...,1
6,"select * from users where id = 1 or 1#"" ( un...",1
7,' select name from syscolumns where id = ( ...,1
8,select * from users where id = 1 +$+ or 1 = 1...,1
9,"1; ( load_file ( char ( 47,101,116,99,47,112,9...",1


In [5]:
df[df['Label']==0]['Label'].count()

22305

In [6]:
33761-22305

11456

In [7]:
# vectorization of data

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, max_features=4096, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()



In [8]:
posts.shape

(33761, 4096)

In [9]:
posts.shape=(33761,64,64,1)

In [10]:
posts.shape

(33761, 64, 64, 1)

In [11]:
X=posts

In [12]:
y=df['Label']

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
# split train test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
trainX=X_train.copy()
trainX.shape=(X_train.shape[0],trainX.shape[1]*trainX.shape[2])
testX=X_test.copy()
testX.shape=(testX.shape[0],testX.shape[1]*testX.shape[2])


In [55]:
model_1 = LogisticRegression()
model_2 = XGBClassifier()
model_3 = RandomForestClassifier()
model_4 = GaussianNB()
model_5 = SVC(gamma='auto')
model_6 = KNeighborsClassifier(n_neighbors=3)
model_7=GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)

In [60]:
model_4.fit(trainX,y_train)

GaussianNB()

In [61]:
model_6.fit(trainX,y_train)

KNeighborsClassifier(n_neighbors=3)

In [62]:
model_1.fit(trainX,y_train)

LogisticRegression()

In [56]:
model_7.fit(trainX,y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [63]:
pred1=model_4.predict(testX)

In [64]:
pred2=model_6.predict(testX)

In [65]:
pred3=model_1.predict(testX)

In [57]:
pred4=model_7.predict(testX)

In [66]:
final_model = VotingClassifier(
    estimators=[('lr',model_1),('gnb', model_4),('knn', model_6)], voting='hard')

In [67]:
final_model.fit(trainX,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('gnb', GaussianNB()),
                             ('knn', KNeighborsClassifier(n_neighbors=3))])

In [68]:
pred_final = final_model.predict(testX)

In [44]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(final_model, open(filename, 'wb'))

In [45]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [46]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [47]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [48]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

# Accuracy of KNN

In [71]:
accuracy,precision,recall=confusion_matrix(y_test,pred2)

In [72]:
accuracy

0.9582407818747224

# Accuracy of Logistics Regression

In [73]:
accuracy,precision,recall=confusion_matrix(y_test,pred1)
accuracy

0.5467199763068266

# Accuracy of KNN

In [74]:
accuracy,precision,recall=confusion_matrix(y_test,pred3)
accuracy

0.9582407818747224

# Accuracy of our ensemble model

In [75]:
accuracy,precision,recall=confusion_matrix(y_test,pred_final)
accuracy

0.9604620168813861

# Accuracy of Gradient boost ensemble model

In [76]:
accuracy,precision,recall=confusion_matrix(y_test,pred4)
accuracy

0.9564637938693914